# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pytz
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from geopy import distance
import datetime
import tilemapbase
from copy import deepcopy
import pickle as pkl
from PIL import Image
import skimage.measure
import math
import warnings
warnings.filterwarnings('ignore')
import hyperopt

# Parameters

In [2]:
source = 'combined'
sensor = 'pm25'
res_time = '1H'
filepath_root = '/scratch/ab9738/pollution_with_sensors/'
spikes_file = filepath_root+'hotspots/spikes_combined_1H.csv'
time_high_file = filepath_root+'hotspots/hotspots_combined_temporalhigh_1H.pkl'
time_low_file = filepath_root+'hotspots/hotspots_combined_temporallow_1H.pkl'
space_high_file = filepath_root+'hotspots/hotspots_combined_spatialhigh_1H.pkl'
space_low_file = filepath_root+'hotspots/hotspots_combined_spatiallow_1H.pkl'

# Data Loading

In [3]:
filepath_data_kai = filepath_root+'data/kaiterra/kaiterra_fieldeggid_{}_current_panel.csv'.format(res_time)
filepath_data_gov = filepath_root+'data/govdata/govdata_{}_current.csv'.format(res_time)
filepath_locs_kai = filepath_root+'data/kaiterra/kaiterra_locations.csv'
filepath_locs_gov = filepath_root+'data/govdata/govdata_locations.csv'

locs_kai = pd.read_csv(filepath_locs_kai, index_col=[0])
locs_kai['Type'] = 'Kaiterra'
locs_gov = pd.read_csv(filepath_locs_gov, index_col=[0])
locs_gov['Type'] = 'Govt'
locs = pd.merge(locs_kai, locs_gov, how='outer',\
                on=['Monitor ID', 'Latitude', 'Longitude', 'Location', 'Type'], copy=False)
data_kai = pd.read_csv(filepath_data_kai, index_col=[0,1], parse_dates=True)[sensor]
data_gov = pd.read_csv(filepath_data_gov, index_col=[0,1], parse_dates=True)[sensor]
data = pd.concat([data_kai, data_gov], axis=0, copy=False)

start_dt = data.index.levels[1][0]
end_dt = data.index.levels[1][-1]

if start_dt.tzname != 'IST':
        if start_dt.tzinfo is None:
            start_dt = start_dt.tz_localize('UTC')
        start_dt = start_dt.tz_convert(pytz.FixedOffset(330))
    
if end_dt.tzname != 'IST':
    if end_dt.tzinfo is None: 
        end_dt = end_dt.tz_localize('UTC')
    end_dt = end_dt.tz_convert(pytz.FixedOffset(330))

# now, filter through the start and end dates
data.sort_index(inplace=True)
data = data.loc[(slice(None), slice(start_dt, end_dt))]

if(source=='govdata'):
    df = data_gov.unstack(level=0)
elif(source=='kaiterra'):
    df = data_kai.unstack(level=0)
else:
    df = data.unstack(level=0)
distances = pd.read_csv('/scratch/ab9738/pollution_with_sensors/data/combined_distances.csv', index_col=[0])
distances = distances.loc[df.columns, df.columns]
distances[distances == 0] = np.nan

In [4]:
locs

,UDID,Latitude,Longitude,Address,Location,Type
Monitor ID,,,,,,
BB4A,492457f434f94afc865bb4701990bb4a,28.554980,77.194430,Jamun,Hauz Khas Village,Kaiterra
91B8,ea3ceeefd9654dfd9aab41422f7391b8,28.503050,77.185660,Vihara,Chhatapur,Kaiterra
BC46,29b8262425cf4135899cd65b2458bc46,28.632950,77.288700,Segel Design,Preet Vihar,Kaiterra
BFDC,11047d2ddc514f63a12ad4f1ad3bbfdc,28.521083,77.214237,Arundhati,Saket,Kaiterra
D804,f083e8afd43e4727a5eb7f3a1529d804,28.558230,77.208620,EPoD,Yusuf Sarai,Kaiterra
...,...,...,...,...,...,...
Sirifort_CPCB,NaN,28.550425,77.215938,NaN,"Sirifort, New Delhi - CPCB",Govt
SoniaVihar_DPCC,NaN,28.710508,77.249485,NaN,"Sonia Vihar, Delhi - DPCC",Govt
SriAurobindoMarg_DPCC,NaN,28.531346,77.190156,NaN,"Sri Aurobindo Marg, Delhi - DPCC",Govt


# Load Hotspots

In [5]:
with open(time_low_file,'rb') as file:
    thsp_low = pkl.load(file)

In [6]:
with open(time_high_file,'rb') as file:
    thsp_high = pkl.load(file)

In [7]:
with open(space_high_file,'rb') as file:
    shsp_high = pkl.load(file)
with open(space_low_file,'rb') as file:
    shsp_low = pkl.load(file)

# Load Wind Speeds

In [8]:
df_ws = pd.read_csv('/scratch/ab9738/pollution_with_sensors/hotspots/source_apportionment/wind_speeds.csv', parse_dates=True)

df_ws = df_ws.sort_values(['Timestamp']).reset_index(drop=True)

df_ws = df_ws.set_index(pd.DatetimeIndex(df_ws['Timestamp']))

df_ws = df_ws[['u-component', 'v-component']].groupby('Timestamp').mean()

In [9]:
df_ws

,u-component,v-component
Timestamp,,
2018-05-01 00:00:00,-5.912000,3.205333
2018-05-01 06:00:00,-3.213333,5.312000
2018-05-01 12:00:00,-3.637333,-0.202667
2018-05-01 18:00:00,-6.544000,2.458667
2018-05-02 00:00:00,-4.949333,2.280000
...,...,...
2020-10-30 18:00:00,2.400000,-1.920000
2020-10-31 00:00:00,2.560000,-1.093333
2020-10-31 06:00:00,4.853333,-4.106667


# Load Intensity Maps

In [10]:
brick_kilns = np.load('brick_kilns_intensity_80x80.npy')
industries = np.load('industries_intensity_80x80.npy')
power_plants = np.load('power_plants_intensity_80x80.npy')
population_density = np.load('population_density_intensity_80x80.npy')
traffic_06 = np.load('traffic_06_intensity_80x80.npy')
traffic_12 = np.load('traffic_12_intensity_80x80.npy')
traffic_18 = np.load('traffic_18_intensity_80x80.npy')
traffic_00 = np.load('traffic_00_intensity_80x80.npy')

# Gaussian-Plume Model for Point Source

## Core Gaussian-Plume Dispersion Formula

In [11]:
def gaussian_plume(src, dest, intensity, H, wind_speed, alpha, z, a, c, d, f):
    if(intensity>0):
        distance_direction = np.array([dest[1]-src[1], dest[0]-src[0]])/math.sqrt((dest[1]-src[1])**2+(dest[0]-src[0])**2) 
        #reversing as lat=y-axis and long=x-axis
        distance_magnitude = distance.distance(src, dest).meters
        distance_vector = distance_magnitude * distance_direction
        unit_wind_vector = wind_speed/math.sqrt(wind_speed[0]**2 + wind_speed[1]**2)
        wind_magnitude = np.linalg.norm(wind_speed)
        distance_wind = np.dot(distance_vector, unit_wind_vector)
        if(distance_wind<=0):
            return(0.0)
        distance_perpendicular = np.linalg.norm(np.subtract(distance_vector, distance_wind))
        sigma_y = c*math.pow(distance_wind,d)+f
        sigma_z = a*math.pow(distance_wind,0.894)
        concentration = ((alpha*intensity)/(2*math.pi*wind_magnitude*sigma_z*sigma_y))*math.exp(-distance_perpendicular**2/(2*sigma_y**2))*\
        (math.exp(-(z-H)**2/(2*sigma_z**2))+math.exp(-(z+H)**2/(2*sigma_z**2)))
        return(concentration*(10**9))
    else:
        return 0

## Computing concentration at a sensor because of different sources

In [12]:
def compute_concentration(dest, ts, alpha, wind_speed, stack_height=None, z=6.5, a=213, c=459.7, d=2.094, f=-9.6):
    if(stack_height is None):
        stack_height = {'traffic':0, 'brick_kilns':25, 'population_density':10, 'industry':30, 'power_plant':200}
    
    idx_x = int((dest[1]-76.85)/0.01)
    idx_y = int((dest[0]-28.2)/0.01)
    src_radius = 7
    if(pd.Timestamp(ts).hour>3 and pd.Timestamp(ts).hour<9):
        traffic_srcs = traffic_06
    elif(pd.Timestamp(ts).hour>=9 and pd.Timestamp(ts).hour<15):
        traffic_srcs = traffic_12
    elif(pd.Timestamp(ts).hour>=15 and pd.Timestamp(ts).hour<21):
        traffic_srcs = traffic_18
    else:
        traffic_srcs = traffic_00
        
    total_concentration = 0.0
        
    for i in range(idx_x-src_radius, idx_x+src_radius+1):
        for j in range(idx_y-src_radius, idx_y+src_radius+1):
            src = (28.2+(j*0.01)+0.005, 76.85+(i*0.01)+0.005)
            total_concentration = total_concentration \
            + gaussian_plume(src, dest, brick_kilns[i,j], stack_height['brick_kilns'], wind_speed, alpha,z,a,c,d,f)\
            + gaussian_plume(src, dest, industries[i,j], stack_height['industry'], wind_speed, alpha, z,a,c,d,f)\
            + gaussian_plume(src, dest, power_plants[i,j], stack_height['power_plant'], wind_speed, alpha, z,a,c,d,f)\
            + gaussian_plume(src, dest, population_density[i,j], stack_height['population_density'], wind_speed, alpha, z,a,c,d,f)\
            + gaussian_plume(src, dest, traffic_srcs[i,j], stack_height['traffic'], wind_speed, alpha, z,a,c,d,f)
            
    return total_concentration

## Finding all sensors near hotspot

In [13]:
def find_sensors_in_region(sensor):
    coord = (locs.loc[sensor]['Latitude'], locs.loc[sensor]['Longitude'])
    region_radius = 2
    idx_x = int((coord[1]-76.85)/0.01)
    idx_y = int((coord[0]-28.2)/0.01)
    region_idx_l, region_idx_r, region_idx_b, region_idx_t =\
    idx_x-region_radius, idx_x+region_radius+1, idx_y-region_radius, idx_y+region_radius+1
    region_long_l, region_long_r, region_lat_b, region_lat_t =\
    76.85+(region_idx_l*0.01), 76.85+(region_idx_r*0.01), 28.2+(region_idx_b*0.01), 28.2+(region_idx_t*0.01)
    subset_locs = locs[(locs['Latitude']<region_lat_t) & (locs['Latitude']>region_lat_b) &\
                       (locs['Longitude']<region_long_r) & (locs['Longitude']>region_long_l)]
    sensors = np.intersect1d(subset_locs.index.to_numpy(),df.columns)
    return(sensors)

## Find nearest wind speed data

In [14]:
def get_wind_speed_estimate(hsp):
    ts = np.array([hsp[0]]).astype('datetime64[ns]')[0]

    if(pd.Timestamp(ts).hour in [5,11,17,23]):
        ts = ts+np.timedelta64(30,'m')
    else:
        ts = ts-np.timedelta64(30,'m')

    ws = df_ws.loc[ts].values
    ws = ws*(5.0/18)
    return(ws)

## Filtering Hotspots based on time so that wind data is only 30 minutes in deviation.

In [15]:
ws_timestamps = df_ws.index.to_numpy()
hsp_timestamps = np.array(shsp_high)[:,0].astype('datetime64[ns]')

df_hsp_timestamps = pd.DataFrame(hsp_timestamps)

shsp_toexplain = np.array(shsp_high)[df_hsp_timestamps[0].apply(lambda x: True if x.hour in [5,6,11,12,17,18,23,0] else False).to_numpy()]

In [16]:
np.random.seed(42)
np.random.shuffle(shsp_toexplain)
train, test = shsp_toexplain[:int(0.8*len(shsp_toexplain))], shsp_toexplain[int(0.8*len(shsp_toexplain)):]

# MLE Training of Model 

In [42]:
def objective(params):
    
    [alpha, H_traffic, H_brick, H_population, H_industry, H_power, z, stability] = params
    
    H = {'traffic':H_traffic, 'brick_kilns':H_brick, 'population_density':H_population, 'industry':H_industry, 'power_plant':H_power}
    stability_dict = {'A': [213,459.7,2.084,-9.6], 'B':[156,108.2,1.098,2.0], 'C':[104,61.0,0.911,0.0], 'D':[68,44.5,0.516,-13.0],\
                     'E':[50.5,55.4,0.305,-34.0], 'F':[34,62.6,0.180,-48.6]}
    [a,c,d,f] = stability_dict[stability]
    
    obj, N = 0, 0
    for i in tqdm(range(len(train))):

        hsp = train[i]

        wind_speed = get_wind_speed_estimate(hsp)

        sensors = find_sensors_in_region(hsp[1])

        sensors = df.loc[hsp[0]][sensors].dropna().index.to_numpy() 

        for sensor in sensors:
            dest = (locs.loc[sensor]['Latitude'], locs.loc[sensor]['Longitude'])
            ts = hsp[0]
            computed_val = compute_concentration(dest=dest, ts=ts, alpha=alpha, wind_speed=wind_speed, stack_height=H, z=z, \
                                                a=a, c=c, d=d, f=f)
            measured_val = df.loc[hsp[0]][sensor]
            obj = obj + (measured_val-computed_val)**2
            N = N+1
            
    rmse = math.sqrt(obj/N)
    return(rmse)

In [43]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [44]:
space = [hp.uniform('alpha', 1, 1000),hp.uniform('H_traffic',0,3),hp.uniform('H_brick',22,50), hp.uniform('H_population', 0, 17.5),\
        hp.uniform('H_industry', 30, 60), hp.uniform('H_power', 200, 400), hp.uniform('z', 3, 10),\
         hp.choice('stability',['A','B','C','D','E','F'])]

In [ ]:
space_alpha = hp.uniform('alpha', 1, 1000)

In [45]:
trials = Trials()

In [ ]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/1591 [00:00<?, ?it/s]

  0%|          | 1/1591 [00:00<06:44,  3.93it/s]

  0%|          | 2/1591 [00:00<04:38,  5.71it/s]

  0%|          | 3/1591 [00:00<08:36,  3.08it/s]

  0%|          | 5/1591 [00:01<04:47,  5.52it/s]

  0%|          | 6/1591 [00:01<07:10,  3.68it/s]

  1%|          | 8/1591 [00:01<04:38,  5.68it/s]

  1%|          | 9/1591 [00:01<05:06,  5.16it/s]

  1%|          | 10/1591 [00:02<05:28,  4.81it/s]

  1%|          | 11/1591 [00:02<05:08,  5.12it/s]

  1%|          | 13/1591 [00:02<06:08,  4.28it/s]

  1%|          | 15/1591 [00:03<04:48,  5.46it/s]

  1%|1         | 17/1591 [00:03<03:38,  7.21it/s]

  1%|1         | 19/1591 [00:03<03:34,  7.32it/s]

  1%|1         | 20/1591 [00:03<03:34,  7.33it/s]

  1%|1         | 22/1591 [00:03<03:31,  7.41it/s]

  1%|1         | 23/1591 [00:04<03:53,  6.72it/s]

  2%|1         | 25/1591 [00:04<03:00,  8.65it/s]

  2%|1         | 27/1591 [00:04<02:46,  9.41it/s]

  2%|1         | 29/1591 [00:04<03:01,  8.60it/

  1%|          | 1/100 [04:03<6:42:15, 243.79s/trial, best loss: 1085765.6778357213]

  0%|          | 0/1591 [00:00<?, ?it/s]

  0%|          | 1/1591 [00:00<06:38,  3.99it/s]

  0%|          | 2/1591 [00:00<04:32,  5.82it/s]

  0%|          | 3/1591 [00:00<08:33,  3.09it/s]

  0%|          | 5/1591 [00:01<04:48,  5.50it/s]

  0%|          | 6/1591 [00:01<07:12,  3.67it/s]

  1%|          | 8/1591 [00:01<04:38,  5.67it/s]

  1%|          | 9/1591 [00:01<05:07,  5.15it/s]

  1%|          | 10/1591 [00:02<05:30,  4.79it/s]

  1%|          | 11/1591 [00:02<05:10,  5.09it/s]

  1%|          | 13/1591 [00:02<06:06,  4.30it/s]

  1%|          | 15/1591 [00:03<04:47,  5.49it/s]

  1%|1         | 17/1591 [00:03<03:36,  7.25it/s]

  1%|1         | 19/1591 [00:03<03:33,  7.37it/s]

  1%|1         | 20/1591 [00:03<03:33,  7.36it/s]

  1%|1         | 22/1591 [00:03<03:29,  7.47it/s]

  1%|1         | 23/1591 [00:04<03:52,  6.75it/s]

  2%|1         | 25/1591 [00:04<02:59,  8.70it/s]

  2%|1         | 27/1591 [00:04<02:44,  9.49it/s]

  2%|1         | 29/1591 [00:04<03:01,  8.62it/

  2%|▏         | 2/100 [08:08<6:38:38, 244.07s/trial, best loss: 982405.0597007114] 

  0%|          | 0/1591 [00:00<?, ?it/s]

  0%|          | 1/1591 [00:00<06:41,  3.96it/s]

  0%|          | 2/1591 [00:00<04:34,  5.80it/s]

  0%|          | 3/1591 [00:00<08:28,  3.12it/s]

  0%|          | 5/1591 [00:01<04:44,  5.57it/s]

  0%|          | 6/1591 [00:01<07:13,  3.66it/s]

  1%|          | 8/1591 [00:01<04:39,  5.66it/s]

  1%|          | 9/1591 [00:01<05:09,  5.12it/s]

  1%|          | 10/1591 [00:02<05:30,  4.79it/s]

  1%|          | 11/1591 [00:02<05:10,  5.09it/s]

  1%|          | 13/1591 [00:02<06:06,  4.30it/s]

  1%|          | 15/1591 [00:03<04:48,  5.47it/s]

  1%|1         | 17/1591 [00:03<03:37,  7.24it/s]

  1%|1         | 19/1591 [00:03<03:33,  7.36it/s]

  1%|1         | 20/1591 [00:03<03:33,  7.37it/s]

  1%|1         | 22/1591 [00:03<03:30,  7.44it/s]

  1%|1         | 23/1591 [00:04<03:54,  6.70it/s]

  2%|1         | 25/1591 [00:04<03:01,  8.62it/s]

  2%|1         | 27/1591 [00:04<02:45,  9.47it/s]

  2%|1         | 29/1591 [00:04<03:02,  8.57it/

In [18]:
import json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
# with open('best.json', 'w') as fp:
#     json.dump(best, fp, cls=NpEncoder)

In [19]:
with open('best_scr.json', 'r') as fp:
    best = json.load(fp)

In [20]:
best

{'H_brick': 48.45614584870221,
 'H_industry': 52.84011804388569,
 'H_population': 10.792480401259688,
 'H_power': 330.77222421365445,
 'H_traffic': 2.081630370005164,
 'alpha': 2.249990775161532,
 'stability': 0,
 'z': 8.872693035785522}

# Evaluation of Gaussian Plume Model

In [22]:
def evaluate(params):
    
    alpha = params['alpha']
    H_traffic = params['H_traffic']
    H_brick = params['H_brick']
    H_population = params['H_population']
    H_industry = params['H_industry']
    H_power = params['H_power']
    z = params['z']
    stability = ['A','B','C','D','E','F'][params['stability']]
    
    H = {'traffic':H_traffic, 'brick_kilns':H_brick, 'population_density':H_population, 'industry':H_industry, 'power_plant':H_power}
    stability_dict = {'A': [213,459.7,2.084,-9.6], 'B':[156,108.2,1.098,2.0], 'C':[104,61.0,0.911,0.0], 'D':[68,44.5,0.516,-13.0],\
                     'E':[50.5,55.4,0.305,-34.0], 'F':[34,62.6,0.180,-48.6]}
    [a,c,d,f] = stability_dict[stability]
    
    phase = test
    
    obj, N = 0, 0
    explained = 0
    chsp, mhsp = [],[]
    for i in tqdm(range(len(phase))):

        hsp = phase[i]

        wind_speed = get_wind_speed_estimate(hsp)
        sensors = find_sensors_in_region(hsp[1])
        sensors = df.loc[hsp[0]][sensors].dropna().index.to_numpy()
        comp, meas = {},{}

        for sensor in sensors:
            dest = (locs.loc[sensor]['Latitude'], locs.loc[sensor]['Longitude'])
            ts = hsp[0]
            computed_val = compute_concentration(dest=dest, ts=ts, alpha=alpha, wind_speed=wind_speed, stack_height=H, z=z, \
                                                a=a, c=c, d=d, f=f)
            measured_val = df.loc[hsp[0]][sensor]
            obj = obj + (measured_val-computed_val)**2
            N = N+1
            comp[sensor] = computed_val
            meas[sensor] = measured_val
            
        if(np.argmax(np.array(list(comp.values())))==np.argmax(np.array(list(meas.values())))):
            explained = explained + 1
        
        chsp.append(comp)
        mhsp.append(meas)
            
    rmse = math.sqrt(obj/N)
    return(rmse, explained, len(phase), chsp, mhsp)

In [23]:
rmse, explained, total, chsp, mhsp = evaluate(best)

100%|██████████| 398/398 [00:56<00:00,  7.00it/s]


In [24]:
print(rmse, explained, total)

1850.4205728474315 259 398


In [52]:
for i in range(len(chsp)):
    if(max(chsp[i], key=chsp[i].get)!=max(mhsp[i], key=mhsp[i].get)):
        print(chsp[i],mhsp[i])

{'PunjabiBagh_DPCC': 0.03653925136961242, 'Shadipur_CPCB': 5.218332903648698} {'PunjabiBagh_DPCC': 256.0, 'Shadipur_CPCB': 52.0}
{'IHBAS_CPCB': 0.6558204886072476, 'VivekVihar_DPCC': 0.11604847165741952} {'IHBAS_CPCB': 43.05, 'VivekVihar_DPCC': 413.5}
{'2E9C': 5.716829729740098, '4BE7': 0.17452186082184956, 'BFDC': 0.2798301208431094, 'E47A': 11.894710025342764} {'2E9C': 363.6666666666667, '4BE7': 154.66666666666666, 'BFDC': 101.0, 'E47A': 146.41666666666666}
{'IHBAS_CPCB': 0.8684753223146626, 'VivekVihar_DPCC': 0.15889975194194678} {'IHBAS_CPCB': 292.9, 'VivekVihar_DPCC': 508.0}
{'CRRIMathuraRoad_IMD': 1.3432386083831813, 'NehruNagar_DPCC': 0.1428472715077926, 'OkhlaPhase2_DPCC': 12.503414465684141} {'CRRIMathuraRoad_IMD': 863.63, 'NehruNagar_DPCC': 556.5, 'OkhlaPhase2_DPCC': 625.0}
{'PunjabiBagh_DPCC': 93.33776251660716, 'Pusa_DPCC': 0.9038026719255586, 'Pusa_IMD': 0.893624649350169, 'Shadipur_CPCB': 5.617375132358268} {'PunjabiBagh_DPCC': 85.0, 'Pusa_DPCC': 82.5, 'Pusa_IMD': 61.24, 